In [1]:
#!pip install --upgrade pip
!#pip install pandas --quiet
#!pip install numpy --quiet
#!python -m pip install scipy --quiet
#!pip install tensorflow --quiet
#!pip install -U scikit-learn --quiet
#!python -m pip install -U matplotlib --quiet
#!pip install xlrd --quiet
#!pip install openpyxl --quiet
#!pip install xgboost --quiet
#!pip install lightgbm --quiet

'#pip' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Conv1D, Dropout
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import scipy.stats as st
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import re
import math
from sklearn.metrics import mean_squared_error
np.random.seed(7)

In [3]:
# parametros para o método de agrupamento de sequencias
grupo = 12
valor_total = 60

In [4]:
# retorna uma lista dos grupos referente a cada número da sequência
def agrupador(lista,grupo,valor_total):
    resultado = []
    for val in lista:
        for x in range(1,(grupo+1)):
            if val <= ((valor_total/grupo)*x):
                resultado.append(x)
                break
    return resultado

### Pegando resultados da Mega da Virada (2005-2020)

In [5]:
df_virada = pd.read_csv('Virada 2009-2021.csv')
df_virada['Data'] = [str(val).strip().replace('/','-') for val in df_virada['Data'].tolist()]
df_virada['Data'] = pd.to_datetime(df_virada['Data'],dayfirst=True)
df_virada.sort_values(by=['Data'],inplace=True)
print(len(df_virada))

13


In [6]:
len(df_virada)

13

### Pegando resultados das Megas Senas (2021-2022)

In [7]:
df_mega_2021 = pd.read_csv('2021.csv')
df_mega_2022 = pd.read_csv('2022.csv')
df_mega = pd.concat([df_mega_2021,df_mega_2022])
df_mega.reset_index(inplace=True)
df_mega['Data'] = [str(val).strip().replace('/','-') for val in df_mega['Data'].tolist()]
df_mega['Data'] = pd.to_datetime(df_mega['Data'],dayfirst=True)
df_mega.drop(columns=['index'],inplace=True)
df_mega.sort_values(by=['Data'],inplace=True)

In [8]:
#df_mega.set_index('Data',inplace=True)
len(df_mega)

204

### Criando dataframe completo (resultados megas senas e megas da virada)

### Dataframes por sequência em ordem de Sorteio

In [9]:
df_total = pd.concat([df_mega,df_virada])
df_total['Data'] = [str(val).strip().replace('/','-') for val in df_total['Data'].tolist()]
df_total['Data'] = pd.to_datetime(df_total['Data'],dayfirst=True)
df_total = df_total.sort_values(by='Data')
df_mega.set_index('Data',inplace=True)
df_virada.set_index('Data',inplace=True)
df_total.set_index('Data',inplace=True)
print(len(df_total))

217


### Dataframes por sequência em ordem crescente

In [10]:
df_mega_ord = pd.DataFrame([sorted(list(row)) for index, row in df_mega.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'], index=df_mega.index)
df_virada_ord = pd.DataFrame([sorted(list(row))for index, row in df_virada.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_virada.index)
df_total_ord = pd.DataFrame([sorted(list(row))for index, row in df_total.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_total.index)

In [11]:
df_total_ord

,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
Data,,,,,,
2009-12-31,10,27,40,46,49,58
2010-12-31,2,10,34,37,43,50
2011-12-31,3,4,29,36,45,55
2012-12-31,14,32,33,36,41,52
2013-12-31,20,30,36,38,47,53
...,...,...,...,...,...,...
2022-10-18,14,17,18,28,30,44
2022-10-20,1,5,18,49,55,56
2022-10-22,10,14,17,18,23,30


# Grupos de Frequência

### Criando Dataframes de grupos de sequências (Ordem de sorteio)

In [12]:
df_mega_seq = pd.DataFrame([agrupador(list(row),grupo,valor_total) for index, row in df_mega.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_mega.index)
df_virada_seq = pd.DataFrame([agrupador(list(row),grupo,valor_total) for index, row in df_virada.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_virada.index)
df_total_seq = pd.DataFrame([agrupador(list(row),grupo,valor_total) for index, row in df_total.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_total.index)

### Criando Dataframes de grupos de sequências (Ordem de crescente)

In [13]:
df_mega_seq_ord = pd.DataFrame([sorted(agrupador(list(row),grupo,valor_total)) for index, row in df_mega.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_mega.index)
df_virada_seq_ord = pd.DataFrame([sorted(agrupador(list(row),grupo,valor_total)) for index, row in df_virada.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_virada.index)
df_total_seq_ord = pd.DataFrame([sorted(agrupador(list(row),grupo,valor_total)) for index, row in df_total.iterrows()],columns=['bola 1', 'bola 2','bola 3','bola 4','bola 5','bola 6'],index=df_total.index)

In [14]:
#df_total = df_total_ord

### Análise de correlações

In [15]:
# normalizando datasets com o méthodo z-score
norm_mega = df_mega.apply(zscore)
norm_virada = df_virada.apply(zscore)
norm_total = df_total.apply(zscore)

In [16]:
# Analisando as correlações entre colunas do dataframe da megasena
norm_mega.corr()
#Melhores Correlações: 2-3 | 3-4 | 1-2 | 5-6

,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
bola 1,1.000000,-0.038660,-0.094408,-0.019710,0.134793,-0.102663
bola 2,-0.038660,1.000000,-0.153564,-0.062506,-0.080573,0.023834
bola 3,-0.094408,-0.153564,1.000000,-0.036728,-0.030579,0.048576
bola 4,-0.019710,-0.062506,-0.036728,1.000000,-0.017414,-0.053209
bola 5,0.134793,-0.080573,-0.030579,-0.017414,1.000000,0.013190
bola 6,-0.102663,0.023834,0.048576,-0.053209,0.013190,1.000000


In [17]:
# Analisando as correlações entre colunas do dataframe da mega da virada
norm_virada.corr()
#Melhores Correlações: 1-2 | 4-3 | 5-3 | 5-4

,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
bola 1,1.000000,0.196252,0.012690,0.045619,0.022403,0.224844
bola 2,0.196252,1.000000,-0.087129,-0.103377,-0.283062,0.115291
bola 3,0.012690,-0.087129,1.000000,0.408696,-0.241983,0.515805
bola 4,0.045619,-0.103377,0.408696,1.000000,-0.023216,-0.059225
bola 5,0.022403,-0.283062,-0.241983,-0.023216,1.000000,-0.359677
bola 6,0.224844,0.115291,0.515805,-0.059225,-0.359677,1.000000


In [18]:
# Analisando as correlações entre colunas do dataframe da megasena e mega da virada
norm_total.corr()
# Melhores Correlações2-1 | 3-2 2-3 | 4-3 3-4 | 5-4 4-5 | 6-5 5-6

# Numeros selecionados para predição por regressão 2 e 4

,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
bola 1,1.000000,-0.027109,-0.093501,-0.013643,0.133565,-0.080636
bola 2,-0.027109,1.000000,-0.149712,-0.064944,-0.091371,0.030082
bola 3,-0.093501,-0.149712,1.000000,-0.015066,-0.045671,0.074262
bola 4,-0.013643,-0.064944,-0.015066,1.000000,-0.014534,-0.051774
bola 5,0.133565,-0.091371,-0.045671,-0.014534,1.000000,-0.007844
bola 6,-0.080636,0.030082,0.074262,-0.051774,-0.007844,1.000000


In [19]:
absolutos = [[df_total['bola 1'].mean(), df_total['bola 1'].std()],
[df_total['bola 2'].mean(), df_total['bola 2'].std()],
[df_total['bola 3'].mean(), df_total['bola 3'].std()],
[df_total['bola 4'].mean(), df_total['bola 4'].std()],
[df_total['bola 5'].mean(), df_total['bola 5'].std()],
[df_total['bola 6'].mean(), df_total['bola 6'].std()]]

In [20]:
coluna = 'bola 4'

# LSTM

In [21]:
#norm_total.reset_index(drop=True,inplace=True)
norm_total.tail(n=12)

,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
Data,,,,,,
2022-09-24,0.373494,0.832295,-1.677179,-0.243566,-1.267744,0.246108
2022-09-28,-0.500498,0.371707,-1.558841,0.555519,0.735414,-0.687757
2022-10-01,1.189221,0.947442,-0.493801,-0.300643,-1.085638,-1.621622
2022-10-05,-1.665822,1.638324,-0.316294,0.384287,-0.903533,0.654674
2022-10-08,1.014422,1.465603,-0.612139,0.384287,-0.114410,-1.388156
2022-10-13,-0.908362,1.695897,-0.434632,1.240450,1.524537,-1.621622
2022-10-15,-1.491024,1.523177,1.577109,1.582915,0.067695,-1.679989
2022-10-18,-0.966628,-0.146454,-0.730477,-0.757263,-0.053708,0.713040
2022-10-20,-0.733563,1.465603,-1.677179,-1.499271,1.099625,1.355073


In [22]:
look_back = 14 #tamanho da janela de predição
init_value = 20 #a partir desse valor que iremos começar a trabalhar, o resto será desconsiderado

dataframe = pd.DataFrame(norm_total[coluna].tolist(), columns = [coluna])
dataframe = dataframe.loc[len(norm_total)-144:,:]
dataset = dataframe.values
dataset = dataset.astype('float32')


# Divite os dados de treino (2/3) e teste (1/3)
# Note que a divisão não é aleatória, mas sim sequencial
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size-look_back-1:len(dataset),:]

In [23]:
def create_dataset(dataset, look_back=1, std=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]-dataset[i]
        a /= std
        dataX.append(a)
        dataY.append(dataset[i + look_back]-dataset[i + look_back-1])
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
#std = train[:, 0].std()
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# shape is [samples, time steps, features]

trainX = trainX.reshape(-1, look_back, 1)
testX = testX.reshape(-1, look_back, 1)
trainY = trainY / 30
testY = testY / 30

trainX.shape, testX.shape, trainY.shape, testY.shape

((82, 14, 1), (49, 14, 1), (82, 1), (49, 1))

In [24]:
model = Sequential()
model.add(LSTM(32, input_shape=(look_back, 1), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4352      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [25]:
callbacks = [
    ReduceLROnPlateau(patience=10, factor=0.5, verbose=True),
    ModelCheckpoint('best.model', save_best_only=True),
    EarlyStopping(patience=25, verbose=True)
]

history = model.fit(trainX, trainY, epochs=5000, batch_size=12, validation_data=(testX, testY),
                    verbose=0, callbacks=callbacks)

INFO:tensorflow:Assets written to: best.model\assets


INFO:tensorflow:Assets written to: best.model\assets


INFO:tensorflow:Assets written to: best.model\assets


INFO:tensorflow:Assets written to: best.model\assets


INFO:tensorflow:Assets written to: best.model\assets


...

INFO:tensorflow:Assets written to: best.model\assets


INFO:tensorflow:Assets written to: best.model\assets


INFO:tensorflow:Assets written to: best.model\assets


INFO:tensorflow:Assets written to: best.model\assets



Epoch 27: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 37: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 42: early stopping


In [26]:
model.load_weights('best.model')
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

2/2 [==============================] - 0s 4ms/step


In [27]:
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 0.04 RMSE
Test Score: 0.03 RMSE


In [28]:
# Prevendo de fato o próximo número da loteria
lista = np.array(dataframe[coluna].tail(n=12).tolist()).reshape(1,12,1)
testPredict = model.predict(lista)
valor_coluna = numero_bola = int(re.findall(r'\b\d+\b', coluna)[0]) - 1
print('O número que pode ser sorteado é aproximadamente: ',testPredict[0][0] * absolutos[int(valor_coluna)][1] + absolutos[int(valor_coluna)][0])

1/1 [==============================] - 0s 427ms/step
O número que pode ser sorteado é aproximadamente:  31.401434749774612


# XGB e LGBM

In [29]:
database = norm_total
numero_bola = 1
#################################### Não alterar a equação abaixo
numero_bola = numero_bola - 1
ultimo_valor = database.tail(n=1)[coluna]

In [30]:
treino = database.loc[norm_total.index <= '2022-07-01',:].copy()
validacao = database.loc[norm_total.index > '2022-07-01',:].copy()

In [31]:
treino.tail()

,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
Data,,,,,,
2022-06-18,0.140430,-1.182777,1.340434,-0.015256,0.674713,-0.104092
2022-06-22,0.781358,-1.528218,1.577109,-1.270961,0.371204,0.654674
2022-06-25,-0.500498,-1.700938,1.458772,-1.556349,-1.267744,1.238340
2022-06-28,0.140430,0.602001,-1.262996,-1.099729,-0.053708,-1.037957
2022-06-30,0.897890,0.026266,-0.197957,1.525837,0.431906,-1.446523


In [32]:
treino['target'] = treino[coluna].shift(-1)
validacao['target'] = validacao[coluna].shift(-1)

In [33]:
treino.dropna(inplace=True)
validacao.dropna(inplace=True)

In [34]:
X_treino = treino.loc[:,[coluna]].values
y_treino = treino.loc[:,[coluna]].values
X_validacao = treino.loc[:,['target']].values
y_validacao = validacao.loc[:,['target']].values

### Predição com XGB

In [35]:
modelo_xgb = XGBRegressor(objective='reg:squarederror',n_estimators=1000)
modelo_xgb.fit(X_treino,y_treino)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [36]:
predicao = modelo_xgb.predict(pd.Series(y_validacao[-1]))

In [37]:
print(f'O número que pode ser sorteado para a {coluna} é aproximadamente: ',predicao[0] * absolutos[numero_bola][1] + absolutos[numero_bola][0])

O número que pode ser sorteado para a bola 4 é aproximadamente:  27.379526093780395


### Predição com LGBM

In [38]:
lgb = LGBMRegressor()
lgb.fit(X_treino,y_treino.ravel())

LGBMRegressor()

In [39]:
predicao = lgb.predict(y_validacao[-1].reshape(-1,1))
print(f'O número que pode ser sorteado para a {coluna} é aproximadamente: ',predicao[0] * absolutos[numero_bola][1] + absolutos[numero_bola][0])

O número que pode ser sorteado para a bola 4 é aproximadamente:  28.003881171694736


## Regressão

### Aquisição de dados e configurações

In [40]:
# prever a quarta bola com base nas bolas 3 e 5 que são as melhores correlacionadas
X_4 = norm_total[['bola 3','bola 5']]
y_4 = norm_total[['bola 4']]

# prever a sexta bola com base nas bolas 1 e 5 que são as melhores correlacionadas
X_2 = norm_total[['bola 1','bola 3']]
y_2 = norm_total[['bola 2']]

#Mude o número dos inputs caso queira prever outra sequência
X,y = X_4,y_4
# Aqui são os valores de X que iremos inputar no preditor para pegarmos os valores de y, ou seja das sequencias 4 ou 6
bola_1 = 22 #valor para predizer a bola 6
bola_3 = 19 #valor para predizer a bola 4
bola_5 = 17 #valor para predizer a bola 6 e 4
predict_x_inputs = [bola_3,bola_5]

numero_bola = int(re.findall(r'\b\d+\b', y.columns[0])[0]) - 1
coluna = y.columns[0]

#Criação de processos para treino e testes de modelos
X_train,X_test,y_train,y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.3, random_state=42)

### Treino e Predição Random Forest

In [41]:
regr = RandomForestRegressor()
regr.fit(X.to_numpy(), y.to_numpy().ravel())
predicoes_analise = regr.predict(X_test)
mse = mean_squared_error(y_test,predicoes_analise)
print(f'O rmse é de: {math.sqrt(mse)}')

O rmse é de: 0.45964984436498785


In [42]:
predicao = regr.predict([predict_x_inputs])
print(f'O número que pode ser sorteado para a {coluna} é aproximadamente: ',predicao[0] * absolutos[numero_bola][1] + absolutos[numero_bola][0])

O número que pode ser sorteado para a bola 4 é aproximadamente:  28.83438778185825


### Treino e Predição XGB

In [43]:
clf = XGBRegressor(eval_metric='rmsle')

In [44]:
clf.fit(X,y)
predicoes_analise = clf.predict(X_test)
mse = mean_squared_error(y_test,predicoes_analise)
print(f'O rmse é de: {math.sqrt(mse)}')

O rmse é de: 0.15557693015187835


In [45]:
predicao = clf.predict(np.array([predict_x_inputs]))
print(f'O número que pode ser sorteado para a {coluna} é aproximadamente: ',predicao[0] * absolutos[numero_bola][1] + absolutos[numero_bola][0])

O número que pode ser sorteado para a bola 4 é aproximadamente:  21.43261597575716


### Treino e Predição LGBM

In [46]:
new = LGBMRegressor()
new.fit(X,y)
predicoes_analise = new.predict(X_test)
mse = mean_squared_error(y_test,predicoes_analise)
print(f'O rmse é de: {math.sqrt(mse)}')

O rmse é de: 0.8259115883762139


In [47]:
predicao = clf.predict(np.array([predict_x_inputs]))
print(f'O número que pode ser sorteado para a {coluna} é aproximadamente: ',predicao[0] * absolutos[numero_bola][1] + absolutos[numero_bola][0])

O número que pode ser sorteado para a bola 4 é aproximadamente:  21.43261597575716


In [48]:
pip freeze --user > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [54]:
b2 = [37,38,45,46]
b3 = [26,27]
b4 = [54,55,53]
b5 = [42,43]
b6 = [51,52]

for a in b2:
    for b in b3:
        for c in b4:
            for d in b5:
                for e in b6:
                    print(f'8-{a}-{b}-{c}-{d}-{e}')

8-37-26-54-42-51
8-37-26-54-42-52
8-37-26-54-43-51
8-37-26-54-43-52
8-37-26-55-42-51
8-37-26-55-42-52
8-37-26-55-43-51
8-37-26-55-43-52
8-37-26-53-42-51
8-37-26-53-42-52
8-37-26-53-43-51
8-37-26-53-43-52
8-37-27-54-42-51
8-37-27-54-42-52
8-37-27-54-43-51
8-37-27-54-43-52
8-37-27-55-42-51
8-37-27-55-42-52
8-37-27-55-43-51
8-37-27-55-43-52
8-37-27-53-42-51
8-37-27-53-42-52
8-37-27-53-43-51
8-37-27-53-43-52
8-38-26-54-42-51
8-38-26-54-42-52
8-38-26-54-43-51
8-38-26-54-43-52
8-38-26-55-42-51
8-38-26-55-42-52
8-38-26-55-43-51
8-38-26-55-43-52
8-38-26-53-42-51
8-38-26-53-42-52
8-38-26-53-43-51
8-38-26-53-43-52
8-38-27-54-42-51
8-38-27-54-42-52
8-38-27-54-43-51
8-38-27-54-43-52
8-38-27-55-42-51
8-38-27-55-42-52
8-38-27-55-43-51
8-38-27-55-43-52
8-38-27-53-42-51
8-38-27-53-42-52
8-38-27-53-43-51
8-38-27-53-43-52
8-45-26-54-42-51
8-45-26-54-42-52
8-45-26-54-43-51
8-45-26-54-43-52
8-45-26-55-42-51
8-45-26-55-42-52
8-45-26-55-43-51
8-45-26-55-43-52
8-45-26-53-42-51
8-45-26-53-42-52
8-45-26-53-43-